Using SPEXone and OCI data for analyzing the impact of dust plumes on marine organism

SETUP <br>
Load necessary packages and auth to cryocloud

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import dask.array as da
import earthaccess
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from dask.distributed import Client
from matplotlib.patches import Rectangle
from rasterio.transform import from_origin

In [ ]:
client = Client()
auth = earthaccess.login("../login.netrc")

2025-09-14 19:48:27,829 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:43209' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'grid_match-c4ff7609dea692eadcb3a30bbdbe6fc5', 'grid_match-b156a1bf81cdf2dbe40dfc97d8a5dc3c', 'grid_match-92702e06e05b2d2b6d61bc6d606c266d', 'grid_match-c4a1d8b10af5cf1c9e37e7c5ea9d76a3', 'grid_match-320a94f6bcdb1f7cf81e1bd6907b3fd3', 'grid_match-10a4a2da8f901fa355ef3ec3b86afb9a', 'grid_match-e0042ae5a5decb4ee26fe992ec53701a'} (stimulus_id='handle-worker-cleanup-1757879307.8297842')
2025-09-14 19:48:27,832 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:42271' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'grid_match-a62fbfe3fc5d3a39768894ee1da20c65', 'grid_match-6f937504655caf02e00449c90880a30f', 'grid_match-bb370d9ffe552c78c079dbddc4e452c4', 'grid_match-842d3d22ad04ea2f7035e68fd3c211be', 'grid_match-39cb60a8f81bb36ddc58331dd8e9b101',

SETUP: LOAD <br>
Specifying parameter for loading the data

In [ ]:
# --- TIME ---

tspan = ("2024-07-01 00:00", "2024-07-01 06:00")

# --- REGION ---

region = (-60, 0, 5, 20)
region = (-180, -90, 180, 90)
#region =  (-20, 10,  -8, 30) #WSEN

SETUP: TARGET GRID <br>
Parameters for interpolating swath data into a common grid

In [ ]:
# --- TARGET GRID ---
res = 1.0

tgt_lat = np.arange(region[1] + res/2, region[3], res)
tgt_lon = np.arange(region[0] + res/2, region[2], res)

# --- INTERPOLATION METHOD ---
method='linear'

FUNCTIONS <br>
Helpers to interpolate the data

In [ ]:
def time_from_attr(ds):
    """
    Set the start time attribute as a dataset variable.

    Parameters
    ----------
    ds a dataset corresponding to a Level-2 granule
    """
    datetime = ds.attrs["time_coverage_start"].replace("Z", "")
    ds["time"] = ((), np.datetime64(datetime, "ns"))
    ds = ds.set_coords("time")
    return ds

def grid_match(path, instr="OCI"):
    """
    Interpolate swath data to common grid

    Parameters
    ----------
    path to the dataset 
    instr the instrument for 
    """
    from rasterio.warp import Resampling

    # specifiy according to instr
    var = "chlor_a"
    geoloc_group = "navigation_data"
    if instr == "SPX":
        var = "chla"
        geoloc_group = "geolocation_data"

    # open dataset
    with xr.open_datatree(path) as dt:
        # Source data & geolocation
        da  = dt["geophysical_data"][var]
        lon = dt[geoloc_group]["longitude"]
        lat = dt[geoloc_group]["latitude"]

        da = da.rio.set_spatial_dims("pixels_per_line", "number_of_lines")        
        da = da.rio.set_crs("epsg:4326")

        # reproject the data to gridded lon-lat data
        da_out = da.rio.reproject(
            dst_crs='EPSG:4326',
            src_geoloc_array=(
                lon.values,
                lat.values,
            ),
            resampling=Resampling.nearest
        )
        # rename the dimensions
        da_out = da_out.rename({"x": "longitude", "y": "latitude"})
        
        # interpolate to common grid
        da_out = da_out.interp(latitude=tgt_lat, longitude=tgt_lon, method="linear", kwargs={"fill_value": np.nan})
        return da_out


LOAD AND TRANSFORM: OCI

In [ ]:
results = earthaccess.search_data(
    short_name="PACE_OCI_L2_BGC",
    temporal=tspan,
    bounding_box=region
)
paths = earthaccess.open(results)


In [ ]:
kwargs = {"combine": "nested", "concat_dim": "time", "decode_timedelta" : True}
attrs = xr.open_mfdataset(paths, preprocess=time_from_attr, **kwargs)
futures = client.map(
    grid_match,
    paths,
)
from dask.distributed import progress
progress(futures)  # <-- show live progress
chla_OCI = xr.combine_nested(client.gather(futures), concat_dim="time")
chla_OCI["time"] = attrs["time"]

LOAD AND TRANSFORM: SPEXONE

In [ ]:
results = earthaccess.search_data(
    short_name="PACE_SPEXONE_L2_AER_RTAPOCEAN",
    temporal=tspan,
    bounding_box=region
)
paths = earthaccess.open(results)

In [ ]:
kwargs = {"combine": "nested", "concat_dim": "time"}
attrs = xr.open_mfdataset(paths, preprocess=time_from_attr, **kwargs)

futures = client.map(
    grid_match,
    paths,
    instr="SPX"
)
chla_SPX = xr.combine_nested(client.gather(futures), concat_dim="time")
chla_SPX["time"] = attrs["time"]


COMPUTE

In [ ]:
diff = (
    chla_OCI.resample(time="3H").mean()
    - chla_SPX.resample(time="3H").mean()
).mean("time")


PLOT

In [ ]:

# Set up plot with projection
fig, ax = plt.subplots(
    subplot_kw={'projection': ccrs.PlateCarree()},
    figsize=(10, 6)
)

# Plot data
im = diff.plot.imshow(
    ax=ax,
    transform=ccrs.PlateCarree(),  # needed so data is placed correctly
    robust=True,
    cmap="viridis"                 # choose colormap if you like
)

# Add coastlines and features
ax.coastlines(resolution="10m", color="black", linewidth=1)
ax.add_feature(cfeature.BORDERS, linestyle=":")
ax.add_feature(cfeature.LAND, facecolor="lightgray")
ax.add_feature(cfeature.OCEAN, facecolor="lightblue")

# Optional: gridlines
gl = ax.gridlines(draw_labels=True, linestyle="--")
gl.top_labels = False
gl.right_labels = False

plt.title("3-hour mean difference: OCI - SPX")
plt.show()